In [2]:
import pandas as pd

df = pd.read_csv('../df_avec_nutriscore_clean.csv', on_bad_lines='skip')


In [3]:
df.to_parquet('../df_avec_nutriscore_clean.parquet', index=False, engine='pyarrow')


In [4]:
df = pd.read_parquet('../df_avec_nutriscore_clean.parquet', engine='pyarrow')

In [5]:
df.shape

(1268142, 41)

In [6]:
collones_pertinantes = ['nutrition-score-fr_100g',
'energy-kcal_100g',
 'fruits-vegetables-nuts-estimate-from-ingredients_100g',
 'carbohydrates_100g',
 'sugars_100g',
 'fat_100g',
 'saturated-fat_100g',
 'proteins_100g',
 'fiber_100g',
 'trans-fat_100g',
 'sodium_100g',
 'salt_100g',
 'cholesterol_100g']


df_collones_pertinantes = df[collones_pertinantes]

In [7]:
df_collones_pertinantes

,nutrition-score-fr_100g,energy-kcal_100g,fruits-vegetables-nuts-estimate-from-ingredients_100g,carbohydrates_100g,sugars_100g,fat_100g,saturated-fat_100g,proteins_100g,fiber_100g,trans-fat_100g,sodium_100g,salt_100g,cholesterol_100g
0,15,324.0,0.000000,13.0,9.00,12.0,10.5,23.0,36.000000,0.0,0.120,0.30,0.00
1,11,376.0,50.000000,6.7,1.30,1.6,0.2,82.0,2.900000,NaN,0.680,1.70,NaN
2,4,363.0,NaN,25.0,0.98,11.0,2.0,22.0,9.000000,0.0,0.380,0.95,0.01
3,4,1.0,0.000000,1.0,1.00,1.0,1.0,1.0,1.000000,NaN,0.400,1.00,NaN
4,6,358.0,0.000000,6.7,1.70,2.0,0.5,76.0,10.714286,NaN,0.600,1.50,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268137,17,333.0,0.000000,30.9,10.40,20.2,6.0,7.4,NaN,NaN,0.440,1.10,NaN
1268138,13,NaN,12.015256,35.0,24.00,7.6,4.8,2.6,NaN,NaN,0.112,0.28,NaN
1268139,1,131.0,0.000000,0.0,0.00,5.0,2.3,21.5,NaN,NaN,0.072,0.18,NaN
1268140,-9,595.0,NaN,16.3,0.00,51.0,6.4,29.8,8.500000,NaN,0.160,0.40,NaN


In [12]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib

features = df_collones_pertinantes.drop(columns=['nutrition-score-fr_100g']).columns.to_list()
target = 'nutrition-score-fr_100g'

X = df_collones_pertinantes[features]
y = df_collones_pertinantes[target]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = lgb.LGBMRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
y_pred = model.predict(X_test)

# Évaluation
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

joblib.dump(model, '../models/model_nutriscore_light_gbm_regressor.pkl')


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0,042344 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3057
[LightGBM] [Info] Number of data points in the train set: 1014513, number of used features: 12
[LightGBM] [Info] Start training from score 11,098379
MAE: 1.297835532099793
RMSE: 5.201536732169662
R²: 0.9509732898006208


['../models/model_nutriscore_light_gbm_regressor.pkl']